# Refresh DCM Datasets for Q4 Campaigns

imports

In [9]:
import sys
sys.path.append('../')

In [10]:
# rtf/essence imports
from rtf_utils.dfa_utils import CampaignManagerReport,get_dfa_report,clean_dcm_file
from rtf_utils.gcp_utils import BigQuery,CloudStorage
from essence.analytics.platform import securedcredentials as secure_creds

# std lib imports
import json
import os
from datetime import datetime
import time
import warnings
import logging

settings/config

In [6]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")
os.environ['GOOGLE_CLOUD_PROJECT'] = 'essence-analytics-dwh' # for some reason gsutil forgets my project

In [12]:
# Auth

In [13]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
print("Creds Loaded")

Creds Loaded


function definitions

In [14]:
def clean_dcm_file(filename):
    data = []
    write = False
    with open(filename,'r') as f:
        reader = csv.reader(f, delimiter=',')

        for row in reader:
            if write == True:
                data.append(row)        
            elif row == ['Report Fields']:
                write = True

        if data[-1][0] == 'Grand Total:':
            data.pop()

    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

# Report Stuff!
DCM Report Definitions:

In [15]:
tasks = [
{'name':'classification','profileId':5096586,'reportId':641053372},
{'name':'classification','profileId':5096586,'reportId':641055447},
{'name':'classification','profileId':5096586,'reportId':641053269},
{'name':'geo_report','profileId':5096586,'reportId':641050621},
{'name':'geo_report','profileId':5096586,'reportId':641055789},
{'name':'geo_report','profileId':5096586,'reportId':641046828},
{'name':'video_player_size','profileId':5096586,'reportId':641052858},
{'name':'video_player_size','profileId':5096586,'reportId':641055441},
{'name':'video_player_size','profileId':5096586,'reportId':641053414}]

#### Update report date range and run

In [ ]:
start_date = '2019-10-01'
end_date = '2019-10-20'
reports = []
for task in tasks:    
    report = CampaignManagerReport(credentialsFromVault,profile_id = 5096586,report_id = task['reportId'])
    print("Fetch report meta")
    report.get_report()    
    print("set date")
    report.set_date_range(start_date,end_date)
    print("run_report")    
    report.run_report()
    reports.append(report)

#### Check if Reports are Ready

In [ ]:
import time
for report in check_q:
    #time.sleep(10)
    print(report.get_report_status())

Upload Loop

In [ ]:
files = []
for report in reports:
    #time.sleep(10)
    try:
        filename = report.download_file()
        files.append(filename)
        print("Saved @ {}".format(filename))
    except Exception as e:
        print("bad news hombre")
        print(e)
        continue
    clean_dcm_file(filename) # filters headers and footers

gcs = CloudStorage(credentialsFromVault)       

blob_names = []

for file in files:    
    blob_name = "brand_reporting/DCM/" + file
    gcs.upload_blob('rtf_staging', blob_name, file, mode="filename")
    blob_names.append(blob_name)